<a href="https://colab.research.google.com/github/Burbina200/Proyecto-IMT2118/blob/main/proyecto_geoespaciales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Geo Espaciales

## Integrantes:
- Camila Basulto
- Benjamín Urbina

## Tema:
- Análisis de cobertura de servicios existentes
- Identificación de zonas aptas para construir

Haremos un mapa con zonas aptas para urbanizar, con recomendaciones específicas sobre qué tipo de infraestructura construir en cada una.


# Fuentes
- Geolocalización de cuerpos de bomberos de chile https://datos.gob.cl/dataset/categoria-geoespacial-instalaciones-y-edificaciones/resource/942164c9-6538-478b-b73c-2672691b36c9

- Establecimientos de educación escolar https://www.geoportal.cl/geoportal/catalog/35408/Establecimientos%20Educaci%C3%B3n%20Escolar